# Graph Classification

In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import sys
import torch
from transformers.optimization import get_cosine_schedule_with_warmup
import torch.nn.functional as F
import torch_geometric.transforms as T
from ogb.graphproppred import PygGraphPropPredDataset, Evaluator
from torch_geometric.loader import DataLoader

import os
import random
import pandas as pd
import torch
import torch_geometric.transforms as T
from typing import Optional
import torch
from torch import Tensor
from torch_geometric.data import Data
from torch_geometric.data.datapipes import functional_transform
from torch_geometric.transforms import BaseTransform

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import WebKB
from torch_geometric.datasets import Actor
from torch_geometric.datasets import GNNBenchmarkDataset
from torch_geometric.datasets import TUDataset
from sklearn.metrics import r2_score
from torch_geometric.data import DataLoader
from torch_geometric.datasets import MoleculeNet
from torch_geometric.nn import GCNConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.utils import to_networkx
from torch.nn import Linear
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import random
import pandas as pd

import time
import psutil
import torch
import torch.nn.functional as F
import warnings
warnings.filterwarnings("ignore")

In [19]:
from torch_geometric.utils.num_nodes import maybe_num_nodes
from torch_sparse import spspmm
from torch_sparse import coalesce
from torch_sparse import eye
from torch.nn import Parameter
from torch_scatter import scatter_add
from torch_scatter import scatter_max

from torch_scatter import scatter_add, scatter
from torch_geometric.nn.inits import uniform
from torch_geometric.nn.resolver import activation_resolver
from torch_geometric.nn import GCNConv, GATConv, LEConv, SAGEConv, GraphConv
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp

from dataclasses import dataclass
from typing import Optional

import torch
from torch import Tensor
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops, to_dense_adj, add_self_loops
from typing import Callable, Optional, Union
from torch_sparse import coalesce, transpose
from torch_scatter import scatter
from torch import Tensor

from typing import Callable, Optional, Tuple, Union

import torch
from torch.nn import Module
from torch_scatter import scatter, scatter_add, scatter_min
from torch_sparse import SparseTensor, remove_diag

from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor

Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]


import torch
from torch_scatter import scatter_max, scatter_min

from torch_geometric.typing import Adj, OptTensor, SparseTensor, Tensor


def maximal_independent_set(edge_index: Adj, k: int = 1,
                            perm: OptTensor = None) -> Tensor:
    r"""Returns a Maximal :math:`k`-Independent Set of a graph, i.e., a set of
    nodes (as a :class:`ByteTensor`) such that none of them are :math:`k`-hop
    neighbors, and any node in the graph has a :math:`k`-hop neighbor in the
    returned set.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method follows `Blelloch's Alogirithm
    <https://arxiv.org/abs/1202.3205>`_ for :math:`k = 1`, and its
    generalization by `Bacciu et al. <https://arxiv.org/abs/2208.03523>`_ for
    higher values of :math:`k`.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: :class:`ByteTensor`
    """
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
        device = edge_index.device()
        n = edge_index.size(0)
    else:
        row, col = edge_index[0], edge_index[1]
        device = row.device
        n = edge_index.max().item() + 1

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    mis = torch.zeros(n, dtype=torch.bool, device=device)
    mask = mis.clone()
    min_rank = rank.clone()

    while not mask.all():
        for _ in range(k):
            min_neigh = torch.full_like(min_rank, fill_value=n)
            scatter_min(min_rank[row], col, out=min_neigh)
            torch.minimum(min_neigh, min_rank, out=min_rank)  # self-loops

        mis = mis | torch.eq(rank, min_rank)
        mask = mis.clone().byte()

        for _ in range(k):
            max_neigh = torch.full_like(mask, fill_value=0)
            scatter_max(mask[row], col, out=max_neigh)
            torch.maximum(max_neigh, mask, out=mask)  # self-loops

        mask = mask.to(dtype=torch.bool)
        min_rank = rank.clone()
        min_rank[mask] = n

    return mis

def maximal_independent_set_cluster(edge_index: Adj, k: int = 1,
                                    perm: OptTensor = None) -> PairTensor:
    r"""Computes the Maximal :math:`k`-Independent Set (:math:`k`-MIS)
    clustering of a graph, as defined in `"Generalizing Downsampling from
    Regular Data to Graphs" <https://arxiv.org/abs/2208.03523>`_.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method returns both the :math:`k`-MIS and the clustering, where the
    :math:`c`-th cluster refers to the :math:`c`-th element of the
    :math:`k`-MIS.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: (:class:`ByteTensor`, :class:`LongTensor`)
    """
    mis = maximal_independent_set(edge_index=edge_index, k=k, perm=perm)
    n, device = mis.size(0), mis.device

    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
    else:
        row, col = edge_index[0], edge_index[1]

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    min_rank = torch.full((n, ), fill_value=n, dtype=torch.long, device=device)
    rank_mis = rank[mis]
    min_rank[mis] = rank_mis

    for _ in range(k):
        min_neigh = torch.full_like(min_rank, fill_value=n)
        scatter_min(min_rank[row], col, out=min_neigh)
        torch.minimum(min_neigh, min_rank, out=min_rank)

    _, clusters = torch.unique(min_rank, return_inverse=True)
    perm = torch.argsort(rank_mis)
    return mis, perm[clusters]


class KMISPooling(Module):

    _heuristics = {None, 'greedy', 'w-greedy'}
    _passthroughs = {None, 'before', 'after'}
    _scorers = {
        'linear',
        'random',
        'constant',
        'canonical',
        'first',
        'last',
    }

    def __init__(self, in_channels: Optional[int] = None, k: int = 1,
                 scorer: Union[Scorer, str] = 'linear',
                 score_heuristic: Optional[str] = 'greedy',
                 score_passthrough: Optional[str] = 'before',
                 aggr_x: Optional[Union[str, Aggregation]] = None,
                 aggr_edge: str = 'sum',
                 aggr_score: Callable[[Tensor, Tensor], Tensor] = torch.mul,
                 remove_self_loops: bool = True) -> None:
        super(KMISPooling, self).__init__()
        assert score_heuristic in self._heuristics, \
            "Unrecognized `score_heuristic` value."
        assert score_passthrough in self._passthroughs, \
            "Unrecognized `score_passthrough` value."

        if not callable(scorer):
            assert scorer in self._scorers, \
                "Unrecognized `scorer` value."

        self.k = k
        self.scorer = scorer
        self.score_heuristic = score_heuristic
        self.score_passthrough = score_passthrough

        self.aggr_x = aggr_x
        self.aggr_edge = aggr_edge
        self.aggr_score = aggr_score
        self.remove_self_loops = remove_self_loops

        if scorer == 'linear':
            assert self.score_passthrough is not None, \
                "`'score_passthrough'` must not be `None`" \
                " when using `'linear'` scorer"

            self.lin = Linear(in_features=in_channels, out_features=1)

    def _apply_heuristic(self, x: Tensor, adj: SparseTensor) -> Tensor:
        if self.score_heuristic is None:
            return x

        row, col, _ = adj.coo()
        x = x.view(-1)

        if self.score_heuristic == 'greedy':
            k_sums = torch.ones_like(x)
        else:
            k_sums = x.clone()

        for _ in range(self.k):
            scatter_add(k_sums[row], col, out=k_sums)

        return x / k_sums

    def _scorer(self, x: Tensor, edge_index: Adj, edge_attr: OptTensor = None,
                batch: OptTensor = None) -> Tensor:
        if self.scorer == 'linear':
            return self.lin(x).sigmoid()

        if self.scorer == 'random':
            return torch.rand((x.size(0), 1), device=x.device)

        if self.scorer == 'constant':
            return torch.ones((x.size(0), 1), device=x.device)

        if self.scorer == 'canonical':
            return -torch.arange(x.size(0), device=x.device).view(-1, 1)

        if self.scorer == 'first':
            return x[..., [0]]

        if self.scorer == 'last':
            return x[..., [-1]]

        return self.scorer(x, edge_index, edge_attr, batch)


    def forward(self, x: Tensor, edge_index: Adj,
                edge_attr: OptTensor = None,
                batch: OptTensor = None) \
            -> Tuple[Tensor, Adj, OptTensor, OptTensor, Tensor, Tensor]:
        """"""
        edge_index = edge_index.long()
        adj, n = edge_index, x.size(0)

        if not isinstance(edge_index, SparseTensor):
            adj = SparseTensor.from_edge_index(edge_index, edge_attr, (n, n))

        score = self._scorer(x, edge_index, edge_attr, batch)
        updated_score = self._apply_heuristic(score, adj)
        perm = torch.argsort(updated_score.view(-1), 0, descending=True)

        mis, cluster = maximal_independent_set_cluster(adj, self.k, perm)

        row, col, val = adj.coo()
        c = mis.sum()

        if val is None:
            val = torch.ones_like(row, dtype=torch.float)

        adj = SparseTensor(row=cluster[row], col=cluster[col], value=val,
                           is_sorted=False,
                           sparse_sizes=(c, c)).coalesce(self.aggr_edge)

        if self.remove_self_loops:
            adj = remove_diag(adj)

        if self.score_passthrough == 'before':
            x = self.aggr_score(x, score)

        if self.aggr_x is None:
            x = x[mis]
        elif isinstance(self.aggr_x, str):
            x = scatter(x, cluster, dim=0, dim_size=mis.sum(),
                        reduce=self.aggr_x)
        else:
            x = self.aggr_x(x, cluster, dim_size=c)

        if self.score_passthrough == 'after':
            x = self.aggr_score(x, score[mis])

        if isinstance(edge_index, SparseTensor):
            edge_index, edge_attr = adj, None

        else:
            row, col, edge_attr = adj.coo()
            edge_index = torch.stack([row, col])

        if batch is not None:
            batch = batch[mis]


        return x, edge_index, edge_attr, batch, mis, cluster

    def __repr__(self):
        if self.scorer == 'linear':
            channels = f"in_channels={self.lin.in_channels}, "
        else:
            channels = ""

        return f'{self.__class__.__name__}({channels}k={self.k})'

### PROTEINS

In [18]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data/Zeyu/Pooling/"

dataset_sparse = TUDataset(root=data_path, name="PROTEINS", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=1, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=1, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 25.64 seconds
Var Time: 0.45 seconds
Average Memory: 144.00 MB
Average Best Val Acc: 0.7430
Std Best Test Acc: 0.0101
Average Test Acc: 0.7063


### NCI1

In [20]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data/Zeyu/Pooling/"

dataset_sparse = TUDataset(root=data_path, name="NCI1", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 76.19 seconds
Var Time: 0.46 seconds
Average Memory: 100.67 MB
Average Best Val Acc: 0.7587
Std Best Test Acc: 0.0219
Average Test Acc: 0.7315


### NCI109

In [21]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data/Zeyu/Pooling/"

dataset_sparse = TUDataset(root=data_path, name="NCI109", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Average Time: 77.94 seconds
Var Time: 4.55 seconds
Average Memory: 102.67 MB
Average Best Val Acc: 0.7426
Std Best Test Acc: 0.0110
Average Test Acc: 0.7317


### MUTAG

In [6]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 150
data_path = "/data/Zeyu/Pooling/1"

dataset_sparse = TUDataset(root=data_path, name="MUTAG", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=1, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=1, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 156 for seed 42
Early stopping at epoch 165 for seed 43
Early stopping at epoch 163 for seed 44
Average Time: 7.32 seconds
Var Time: 0.03 seconds
Average Memory: 32.67 MB
Average Best Val Acc: 0.8690
Std Best Test Acc: 0.0430
Average Test Acc: 0.8046


### DD

In [11]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="DD", pre_filter=lambda data: data.num_nodes <= max_nodes, use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 151 for seed 44
Average Time: 25.64 seconds
Var Time: 9.66 seconds
Average Memory: 575.33 MB
Average Best Val Acc: 0.7658
Std Best Test Acc: 0.0170
Average Test Acc: 0.7057


In [20]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="IMDB-BINARY", transform=T.Compose([T.OneHotDegree(136)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 170 for seed 42
Early stopping at epoch 175 for seed 44
Average Time: 27.48 seconds
Var Time: 3.11 seconds
Average Memory: 138.67 MB
Average Best Val Acc: 0.7778
Std Best Test Acc: 0.0362
Average Test Acc: 0.7289


### IMDB-MULTI

In [23]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
max_nodes = 500
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="IMDB-MULTI", transform=T.Compose([T.OneHotDegree(88)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=1, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=1, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        #print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Early stopping at epoch 173 for seed 42
Early stopping at epoch 190 for seed 43
Average Time: 39.87 seconds
Var Time: 5.52 seconds
Average Memory: 131.33 MB
Average Best Val Acc: 0.5200
Std Best Test Acc: 0.0247
Average Test Acc: 0.4696


### COLLAB

In [14]:
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.data import DenseDataLoader
data_path = "/data/Zeyu/Pooling"

dataset_sparse = TUDataset(root=data_path, name="COLLAB", transform=T.Compose([T.OneHotDegree(491)]), use_node_attr=True)
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ASAPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from torch_geometric.nn import BatchNorm

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_classes):
        super(HierarchicalGCN_KMIS, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool1 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels)
        self.pool2 = KMISPooling(64, k=2, aggr_x='sum')
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = torch.nn.BatchNorm1d(out_channels)

        self.lin1 = torch.nn.Linear(out_channels, 32)
        self.lin2 = torch.nn.Linear(32, num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch

        # First GCN and pooling layer
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        #x = self.bn1(x)
        x, edge_index, _, batch, _, _ = self.pool1(x, edge_index, batch=batch)

        # Second GCN and pooling layer
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        #x = self.bn2(x)
        x, edge_index, _, batch, _, _ = self.pool2(x, edge_index, batch=batch)

        # Third GCN layer
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        #x = self.bn3(x)

        # Mean pooling over the nodes
        x, mask = to_dense_batch(x, batch)
        x = x.mean(dim=1)

        # Fully connected layers
        x = self.lin1(x).relu()
        x = self.lin2(x)
        return F.log_softmax(x, dim=-1)


num_classes = dataset_sparse.num_classes
in_channels = dataset_sparse.num_features

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()
    total_loss = 0
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * data.num_graphs
    return total_loss / len(train_loader.dataset)

def test(loader):
    model.eval()
    correct = 0
    for data in loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += (pred == data.y).sum().item()
    return correct / len(loader.dataset)

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seeds = [42, 43, 44]
times = []
memories = []
best_val_accs = []
best_test_accs = []

early_stop_patience = 150
tolerance = 0.0001

for seed in seeds:
    set_seed(seed)

    dataset_sparse = dataset_sparse.shuffle()

    train_ratio = 0.7
    val_ratio = 0.15
    val_ratio = 0.15
    # Calculate the sizes of each subset
    num_total = len(dataset_sparse)
    num_train = int(num_total * train_ratio)
    num_val = int(num_total * val_ratio)
    num_test = num_total - num_train - num_val
    train_dataset = dataset_sparse[:num_train]
    val_dataset = dataset_sparse[num_train:num_train + num_val]
    test_dataset = dataset_sparse[num_train + num_val:]
    train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
    valid_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

    model = HierarchicalGCN_KMIS(in_channels=dataset_sparse.num_features, hidden_channels=64,out_channels=64, num_classes=dataset_sparse.num_classes).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

    start_time = time.time()

    best_val_acc = 0
    epochs_no_improve = 0

    for epoch in range(1, 201):
        loss = train()
        val_acc = test(valid_loader)
        test_acc = test(test_loader)
        if val_acc > best_val_acc + tolerance:
            best_val_acc = val_acc
            best_test_acc = test_acc
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        print(f'Seed: {seed}, Epoch: {epoch:03d}, Loss: {loss:.4f}, Val Acc: {val_acc:.4f}, Test Acc: {test_acc:.4f}')

        if epochs_no_improve >= early_stop_patience:
            print(f'Early stopping at epoch {epoch} for seed {seed}')
            break

    end_time = time.time()
    total_time = end_time - start_time
    memory_allocated = torch.cuda.memory_reserved(device) / (1024 ** 2)  # Convert to MB

    times.append(total_time)
    memories.append(memory_allocated)
    best_val_accs.append(best_val_acc)
    best_test_accs.append(best_test_acc)

    torch.cuda.empty_cache()

print(f'Average Time: {np.mean(times):.2f} seconds')
print(f'Var Time: {np.var(times):.2f} seconds')
print(f'Average Memory: {np.mean(memories):.2f} MB')
print(f'Average Best Val Acc: {np.mean(best_val_accs):.4f}')
print(f'Std Best Test Acc: {np.std(best_test_accs):.4f}')
print(f'Average Test Acc: {np.mean(best_test_accs):.4f}')

Seed: 42, Epoch: 001, Loss: 1.0305, Val Acc: 0.6147, Test Acc: 0.6067
Seed: 42, Epoch: 002, Loss: 0.9300, Val Acc: 0.6480, Test Acc: 0.6440
Seed: 42, Epoch: 003, Loss: 0.8238, Val Acc: 0.6667, Test Acc: 0.6587
Seed: 42, Epoch: 004, Loss: 0.7196, Val Acc: 0.6813, Test Acc: 0.6733
Seed: 42, Epoch: 005, Loss: 0.6395, Val Acc: 0.7080, Test Acc: 0.6827
Seed: 42, Epoch: 006, Loss: 0.5887, Val Acc: 0.7013, Test Acc: 0.6947
Seed: 42, Epoch: 007, Loss: 0.5590, Val Acc: 0.7107, Test Acc: 0.6947
Seed: 42, Epoch: 008, Loss: 0.5398, Val Acc: 0.6773, Test Acc: 0.6733
Seed: 42, Epoch: 009, Loss: 0.5180, Val Acc: 0.7547, Test Acc: 0.7253
Seed: 42, Epoch: 010, Loss: 0.4898, Val Acc: 0.7760, Test Acc: 0.7333
Seed: 42, Epoch: 011, Loss: 0.4732, Val Acc: 0.7627, Test Acc: 0.7333
Seed: 42, Epoch: 012, Loss: 0.4609, Val Acc: 0.7773, Test Acc: 0.7453
Seed: 42, Epoch: 013, Loss: 0.4544, Val Acc: 0.7600, Test Acc: 0.7387
Seed: 42, Epoch: 014, Loss: 0.4494, Val Acc: 0.7787, Test Acc: 0.7560
Seed: 42, Epoch: 015

# Graph Regression

### QM7

In [30]:
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm7 --cuda_num -1 --run_times=5 --patience=150 --epochs=500 --pooling='KMIS'

Existed dataset loaded: datasets/processed/qm7.pt

Current dataset: qm7, include 6832 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/500MAE=1549.5618 MAE=1547.0464 MAE=1544.4448 MAE=1542.1294 MAE=1539.6531 MAE=1536.8313 MAE=1531.3667 MAE=1527.2961 MAE=1524.4912 Epoch: 10/500MAE=1520.7839 MAE=1513.5902 MAE=1508.8425 MAE=1503.6182 MAE=1495.1584 MAE=1492.0415 MAE=1484.4242 MAE=1470.2502 MAE=1463.0957 MAE=1451.1276 Epoch: 20/500MAE=1442.1366 MAE=1432.0121 MAE=1428.1873 MAE=1423.5129 MAE=1406.6204 MAE=1398.7534 MAE=1386.5073 MAE=1373.7421 MAE=1367.6544 MAE=1353.0543 Epoch: 30/500MAE=1334.3445 MAE=1326.7229 MAE=1320.0898 MAE=1284.9170 MAE=1279.4819 MAE=1283.1274 MAE=1277.3868 MAE=1250.5691 MAE=1229.1216 MAE=1231.8153 Epoch: 40/500MAE=1196.9365 MAE=1201.6630 MAE=1164.7148 MAE=1161.7800 MAE=1148.6885 MAE=1129.2488 MAE=1131.0776 MAE=1109.1387 MAE=1077.2673 MAE=1087.8845 Epoch: 50/500MAE=1055.1865 MAE=1016.9828 MAE=1028.6208 MAE=991.5339 MAE=971.1177 MAE=932.2491 MAE=943.5845

### QM8

In [32]:
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=qm8 --cuda_num -1 --run_times=5 --patience=10 --epochs=150 --pooling='KMIS'

Existed dataset loaded: datasets/processed/qm8.pt

Current dataset: qm8, include 21783 molecules and 12 regression tasks

Splitting, finish 1/1  
Epoch: 1/150MAE=0.1314 MAE=0.1341 MAE=0.1101 MAE=0.0809 MAE=0.0622 MAE=0.0515 MAE=0.0466 MAE=0.0420 MAE=0.0390 Epoch: 10/150MAE=0.0373 MAE=0.0357 MAE=0.0359 MAE=0.0323 MAE=0.0307 MAE=0.0300 MAE=0.0289 MAE=0.0281 MAE=0.0282 MAE=0.0269 Epoch: 20/150MAE=0.0255 MAE=0.0253 MAE=0.0248 MAE=0.0258 MAE=0.0238 MAE=0.0267 MAE=0.0277 MAE=0.0237 MAE=0.0231 MAE=0.0225 Epoch: 30/150MAE=0.0218 MAE=0.0220 MAE=0.0216 MAE=0.0212 MAE=0.0208 MAE=0.0210 MAE=0.0206 MAE=0.0203 MAE=0.0233 MAE=0.0207 Epoch: 40/150MAE=0.0201 MAE=0.0224 MAE=0.0210 MAE=0.0199 MAE=0.0198 MAE=0.0228 MAE=0.0218 MAE=0.0274 MAE=0.0213 MAE=0.0191 Epoch: 50/150MAE=0.0188 MAE=0.0206 MAE=0.0193 MAE=0.0199 MAE=0.0197 MAE=0.0186 MAE=0.0181 MAE=0.0183 MAE=0.0182 MAE=0.0184 Epoch: 60/150MAE=0.0181 MAE=0.0184 MAE=0.0181 MAE=0.0181 MAE=0.0179 MAE=0.0183 MAE=0.0179 MAE=0.0184 MAE=0.0182 MAE=0.0178 Epoch

### BACE

In [21]:
!python /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=bace --cuda_num -1 --run_times=5 --patience=20 --epochs=150 --pooling='KMIS'

Existed dataset loaded: datasets/processed/bace.pt

Current dataset: bace, include 1513 molecules and 1 classification tasks

Splitting, finish 1/1  
Epoch: 1/150Epoch: 10/150Epoch: 20/150
********************1's fold 1's run over********************
AUROC: 0.829 +/- 0.000
AUPRC: 0.786 +/- 0.000

Epoch: 1/150Epoch: 10/150Epoch: 20/150
********************1's fold 2's run over********************
AUROC: 0.837 +/- 0.008
AUPRC: 0.791 +/- 0.005

Epoch: 1/150Epoch: 10/150Epoch: 20/150Epoch: 30/150
********************1's fold 3's run over********************
AUROC: 0.852 +/- 0.022
AUPRC: 0.804 +/- 0.019

Epoch: 1/150Epoch: 10/150Epoch: 20/150
********************1's fold 4's run over********************
AUROC: 0.855 +/- 0.020
AUPRC: 0.811 +/- 0.020

Epoch: 1/150Epoch: 10/150Epoch: 20/150
********************1's fold 5's run over********************
AUROC: 0.851 +/- 0.020
AUPRC: 0.810 +/- 0.018



### ESOL

In [24]:
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=esol --run_times=5 --patience=20 --epochs=150 --pooling='KMIS'

Existed dataset loaded: datasets/processed/esol.pt

Current dataset: esol, include 1127 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=1.5328 RMSE=1.3712 RMSE=1.1044 RMSE=1.1627 RMSE=1.1003 RMSE=1.0397 RMSE=1.0112 RMSE=0.9336 RMSE=0.9707 Epoch: 10/150RMSE=1.0671 RMSE=0.9404 RMSE=1.0075 RMSE=0.9318 RMSE=0.9162 RMSE=0.9720 RMSE=0.9732 RMSE=0.9573 RMSE=0.9403 RMSE=0.9709 Epoch: 20/150RMSE=0.9256 RMSE=0.9589 RMSE=0.9281 RMSE=0.9137 RMSE=0.9101 RMSE=0.8870 RMSE=0.9015 RMSE=0.8946 RMSE=0.9023 RMSE=0.8860 Epoch: 30/150RMSE=0.8911 RMSE=0.8926 RMSE=0.8933 RMSE=0.8995 RMSE=0.8965 RMSE=0.8918 RMSE=0.8967 RMSE=0.8899 RMSE=0.8960 RMSE=0.8949 Epoch: 40/150RMSE=0.8958 RMSE=0.8972 RMSE=0.8922 RMSE=0.8936 RMSE=0.8936 RMSE=0.8944 RMSE=0.9242 
********************1's fold 1's run over********************
RMSE: 0.924 +/- 0.000

Epoch: 1/150RMSE=2.0167 RMSE=1.5934 RMSE=1.4871 RMSE=1.4690 RMSE=1.3988 RMSE=1.6020 RMSE=1.1155 RMSE=1.0913 RMSE=1.0293 Epoch: 10/150RMSE=1.1555 RMSE=1.

### Freesolv

In [27]:
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=freesolv --run_times=5 --patience=20 --epochs=150 --pooling='KMIS'

Existed dataset loaded: datasets/processed/freesolv.pt

Current dataset: freesolv, include 639 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=5.0398 RMSE=4.2502 RMSE=3.4489 RMSE=2.2976 RMSE=1.8993 RMSE=2.1993 RMSE=1.8940 RMSE=1.6753 RMSE=1.9713 Epoch: 10/150RMSE=1.8214 RMSE=2.1092 RMSE=2.0908 RMSE=1.6954 RMSE=1.7761 RMSE=1.6625 RMSE=1.6235 RMSE=1.6587 RMSE=1.5381 RMSE=1.7271 Epoch: 20/150RMSE=1.6846 RMSE=1.6177 RMSE=1.5772 RMSE=1.6836 RMSE=1.6554 RMSE=1.6195 RMSE=1.6511 RMSE=1.6449 RMSE=1.6653 RMSE=1.7070 Epoch: 30/150RMSE=1.7297 RMSE=1.7472 RMSE=1.7410 RMSE=1.7282 RMSE=1.7363 RMSE=1.7367 RMSE=1.7313 RMSE=1.7279 RMSE=1.7427 RMSE=1.6409 
********************1's fold 1's run over********************
RMSE: 1.641 +/- 0.000

Epoch: 1/150RMSE=5.1072 RMSE=4.2130 RMSE=3.1025 RMSE=2.6589 RMSE=2.4019 RMSE=2.0466 RMSE=2.1816 RMSE=2.2785 RMSE=1.9813 Epoch: 10/150RMSE=1.9025 RMSE=2.1090 RMSE=1.8004 RMSE=1.6068 RMSE=1.5191 RMSE=1.6012 RMSE=1.5312 RMSE=1.5982 RMSE=1.5553 R

### Lipophilicity

In [29]:
!python -W ignore /data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/run_regression.py --dataset=lipo --run_times=5 --patience=20 --epochs=150 --pooling='KMIS'

Existed dataset loaded: datasets/processed/lipo.pt

Current dataset: lipo, include 4200 molecules and 1 regression tasks

Splitting, finish 1/1  
Epoch: 1/150RMSE=1.3965 RMSE=1.1359 RMSE=1.0442 RMSE=0.9356 RMSE=0.9455 RMSE=0.9673 RMSE=0.8954 RMSE=0.9330 RMSE=1.1580 Epoch: 10/150RMSE=0.9578 RMSE=0.9415 RMSE=0.8825 RMSE=0.8082 RMSE=0.8100 RMSE=0.8197 RMSE=0.8417 RMSE=0.8081 RMSE=0.7925 RMSE=0.7617 Epoch: 20/150RMSE=0.7719 RMSE=0.7698 RMSE=0.7524 RMSE=0.7560 RMSE=0.7675 RMSE=0.7570 RMSE=0.7756 RMSE=0.7595 RMSE=0.7557 RMSE=0.7542 Epoch: 30/150RMSE=0.7451 RMSE=0.7749 RMSE=0.7620 RMSE=0.7572 RMSE=0.7619 RMSE=0.7500 RMSE=0.7510 RMSE=0.7545 RMSE=0.7615 RMSE=0.7635 Epoch: 40/150RMSE=0.7494 RMSE=0.7531 RMSE=0.7542 RMSE=0.7520 RMSE=0.7520 RMSE=0.7512 RMSE=0.7508 RMSE=0.7511 RMSE=0.7509 RMSE=0.7501 Epoch: 50/150RMSE=0.7498 RMSE=0.8584 
********************1's fold 1's run over********************
RMSE: 0.858 +/- 0.000

Epoch: 1/150RMSE=1.5931 RMSE=1.2750 RMSE=1.1560 RMSE=1.0585 RMSE=1.0685 RMSE=0.

# Node Classification

In [7]:
from typing import Callable, Optional, Tuple, Union
from torch_geometric.typing import Adj, OptTensor, PairTensor, Tensor
Scorer = Callable[[Tensor, Adj, OptTensor, OptTensor], Tensor]
from torch_sparse import SparseTensor, remove_diag
from torch_geometric.nn.aggr import Aggregation
from torch_geometric.nn.dense import Linear
from torch.nn import Module
from torch_scatter import scatter_max, scatter_min

def maximal_independent_set(edge_index: Adj, k: int = 1,
                            perm: OptTensor = None) -> Tensor:
    r"""Returns a Maximal :math:`k`-Independent Set of a graph, i.e., a set of
    nodes (as a :class:`ByteTensor`) such that none of them are :math:`k`-hop
    neighbors, and any node in the graph has a :math:`k`-hop neighbor in the
    returned set.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method follows `Blelloch's Alogirithm
    <https://arxiv.org/abs/1202.3205>`_ for :math:`k = 1`, and its
    generalization by `Bacciu et al. <https://arxiv.org/abs/2208.03523>`_ for
    higher values of :math:`k`.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: :class:`ByteTensor`
    """
    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
        device = edge_index.device()
        n = edge_index.size(0)
    else:
        row, col = edge_index[0], edge_index[1]
        device = row.device
        n = edge_index.max().item() + 1

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    mis = torch.zeros(n, dtype=torch.bool, device=device)
    mask = mis.clone()
    min_rank = rank.clone()

    while not mask.all():
        for _ in range(k):
            min_neigh = torch.full_like(min_rank, fill_value=n)
            scatter_min(min_rank[row], col, out=min_neigh)
            torch.minimum(min_neigh, min_rank, out=min_rank)  # self-loops

        mis = mis | torch.eq(rank, min_rank)
        mask = mis.clone().byte()

        for _ in range(k):
            max_neigh = torch.full_like(mask, fill_value=0)
            scatter_max(mask[row], col, out=max_neigh)
            torch.maximum(max_neigh, mask, out=mask)  # self-loops

        mask = mask.to(dtype=torch.bool)
        min_rank = rank.clone()
        min_rank[mask] = n

    return mis

def maximal_independent_set_cluster(edge_index: Adj, k: int = 1,
                                    perm: OptTensor = None) -> PairTensor:
    r"""Computes the Maximal :math:`k`-Independent Set (:math:`k`-MIS)
    clustering of a graph, as defined in `"Generalizing Downsampling from
    Regular Data to Graphs" <https://arxiv.org/abs/2208.03523>`_.
    The algorithm greedily selects the nodes in their canonical order. If a
    permutation :obj:`perm` is provided, the nodes are extracted following
    that permutation instead.
    This method returns both the :math:`k`-MIS and the clustering, where the
    :math:`c`-th cluster refers to the :math:`c`-th element of the
    :math:`k`-MIS.
    Args:
        edge_index (Tensor or SparseTensor): The graph connectivity.
        k (int): The :math:`k` value (defaults to 1).
        perm (LongTensor, optional): Permutation vector. Must be of size
            :obj:`(n,)` (defaults to :obj:`None`).
    :rtype: (:class:`ByteTensor`, :class:`LongTensor`)
    """
    mis = maximal_independent_set(edge_index=edge_index, k=k, perm=perm)
    n, device = mis.size(0), mis.device

    if isinstance(edge_index, SparseTensor):
        row, col, _ = edge_index.coo()
    else:
        row, col = edge_index[0], edge_index[1]

    if perm is None:
        rank = torch.arange(n, dtype=torch.long, device=device)
    else:
        rank = torch.zeros_like(perm)
        rank[perm] = torch.arange(n, dtype=torch.long, device=device)

    min_rank = torch.full((n, ), fill_value=n, dtype=torch.long, device=device)
    rank_mis = rank[mis]
    min_rank[mis] = rank_mis

    for _ in range(k):
        min_neigh = torch.full_like(min_rank, fill_value=n)
        scatter_min(min_rank[row], col, out=min_neigh)
        torch.minimum(min_neigh, min_rank, out=min_rank)

    _, clusters = torch.unique(min_rank, return_inverse=True)
    perm = torch.argsort(rank_mis)
    return mis, perm[clusters]


class KMISPooling(Module):

    _heuristics = {None, 'greedy', 'w-greedy'}
    _passthroughs = {None, 'before', 'after'}
    _scorers = {
        'linear',
        'random',
        'constant',
        'canonical',
        'first',
        'last',
    }

    def __init__(self, in_channels: Optional[int] = None, k: int = 1,
                 scorer: Union[Scorer, str] = 'linear',
                 score_heuristic: Optional[str] = 'greedy',
                 score_passthrough: Optional[str] = 'before',
                 aggr_x: Optional[Union[str, Aggregation]] = None,
                 aggr_edge: str = 'sum',
                 aggr_score: Callable[[Tensor, Tensor], Tensor] = torch.mul,
                 remove_self_loops: bool = True) -> None:
        super(KMISPooling, self).__init__()
        assert score_heuristic in self._heuristics, \
            "Unrecognized `score_heuristic` value."
        assert score_passthrough in self._passthroughs, \
            "Unrecognized `score_passthrough` value."

        if not callable(scorer):
            assert scorer in self._scorers, \
                "Unrecognized `scorer` value."

        self.k = k
        self.scorer = scorer
        self.score_heuristic = score_heuristic
        self.score_passthrough = score_passthrough

        self.aggr_x = aggr_x
        self.aggr_edge = aggr_edge
        self.aggr_score = aggr_score
        self.remove_self_loops = remove_self_loops

        if scorer == 'linear':
            assert self.score_passthrough is not None, \
                "`'score_passthrough'` must not be `None`" \
                " when using `'linear'` scorer"

            self.lin = Linear(in_features=in_channels, out_features=1)

    def _apply_heuristic(self, x: Tensor, adj: SparseTensor) -> Tensor:
        if self.score_heuristic is None:
            return x

        row, col, _ = adj.coo()
        x = x.view(-1)

        if self.score_heuristic == 'greedy':
            k_sums = torch.ones_like(x)
        else:
            k_sums = x.clone()

        for _ in range(self.k):
            scatter_add(k_sums[row], col, out=k_sums)

        return x / k_sums

    def _scorer(self, x: Tensor, edge_index: Adj, edge_attr: OptTensor = None,
                batch: OptTensor = None) -> Tensor:
        if self.scorer == 'linear':
            return self.lin(x).sigmoid()

        if self.scorer == 'random':
            return torch.rand((x.size(0), 1), device=x.device)

        if self.scorer == 'constant':
            return torch.ones((x.size(0), 1), device=x.device)

        if self.scorer == 'canonical':
            return -torch.arange(x.size(0), device=x.device).view(-1, 1)

        if self.scorer == 'first':
            return x[..., [0]]

        if self.scorer == 'last':
            return x[..., [-1]]

        return self.scorer(x, edge_index, edge_attr, batch)


    def forward(self, x: Tensor, edge_index: Adj,
                edge_attr: OptTensor = None,
                batch: OptTensor = None) \
            -> Tuple[Tensor, Adj, OptTensor, OptTensor, Tensor, Tensor]:
        """"""
        edge_index = edge_index.long()
        adj, n = edge_index, x.size(0)

        if not isinstance(edge_index, SparseTensor):
            adj = SparseTensor.from_edge_index(edge_index, edge_attr, (n, n))

        score = self._scorer(x, edge_index, edge_attr, batch)
        updated_score = self._apply_heuristic(score, adj)
        perm = torch.argsort(updated_score.view(-1), 0, descending=True)

        mis, cluster = maximal_independent_set_cluster(adj, self.k, perm)

        row, col, val = adj.coo()
        c = mis.sum()

        if val is None:
            val = torch.ones_like(row, dtype=torch.float)

        adj = SparseTensor(row=cluster[row], col=cluster[col], value=val,
                           is_sorted=False,
                           sparse_sizes=(c, c)).coalesce(self.aggr_edge)

        if self.remove_self_loops:
            adj = remove_diag(adj)

        if self.score_passthrough == 'before':
            x = self.aggr_score(x, score)

        if self.aggr_x is None:
            x = x[mis]
        elif isinstance(self.aggr_x, str):
            x = scatter(x, cluster, dim=0, dim_size=mis.sum(),
                        reduce=self.aggr_x)
        else:
            x = self.aggr_x(x, cluster, dim_size=c)

        if self.score_passthrough == 'after':
            x = self.aggr_score(x, score[mis])

        if isinstance(edge_index, SparseTensor):
            edge_index, edge_attr = adj, None

        else:
            row, col, edge_attr = adj.coo()
            edge_index = torch.stack([row, col])

        if batch is not None:
            batch = batch[mis]

        #attn = x
        #select_out = topk(attn, batch)
        perm = perm[mis]


        return x, edge_index, edge_attr, batch, mis, cluster, perm

    def __repr__(self):
        if self.scorer == 'linear':
            channels = f"in_channels={self.lin.in_channels}, "
        else:
            channels = ""

        return f'{self.__class__.__name__}({channels}k={self.k})'

### Cora

In [27]:
dataset_sparse = Planetoid(root="/data/Zeyu/Pooling", name='Cora')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=1, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])
Seed 42, Fold 1 Val Acc: 0.897
Seed 42, Fold 2 Val Acc: 0.900
Seed 42, Fold 3 Val Acc: 0.878
Seed 42, Fold 4 Val Acc: 0.895
Early stopping at epoch 189 for fold 5
Seed 42, Fold 5 Val Acc: 0.911
Seed 42 Results: Mean Val Acc: 0.896, Time: 37.200 seconds, Memory: 0.363 MB, GPU Memory: 603.980 MB
Early stopping at epoch 190 for fold 1
Seed 123, Fold 1 Val Acc: 0.895
Seed 123, Fold 2 Val Acc: 0.900
Seed 123, Fold 3 Val Acc: 0.908
Seed 123, Fold 4 Val Acc: 0.900
Seed 123, Fold 5 Val Acc: 0.885
Seed 123 Results: Mean Val Acc: 0.898, Time: 36.886 seconds, Memory: 0.343 MB, GPU Memory: 603.980 MB
Seed 456, Fold 1 Val Acc: 0.900
Seed 456, Fold 2 Val Acc: 0.871
Seed 456, Fold 3 Val Acc: 0.900
Early stopping at epoch 200 for fold 4
Seed 456, Fold 4 Val Acc: 0.898
Seed 456, Fold 5 Val Acc: 0.909
Seed 456 Results: Mean Val Acc: 0.896, Time: 35.205 seconds, Memory: 0.345 MB, GPU Memory: 603.98

### CiteSeer

In [26]:
dataset_sparse = Planetoid(root="/data/Zeyu/Pooling", name='CiteSeer')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=2, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])
Early stopping at epoch 191 for fold 1
Seed 42, Fold 1 Val Acc: 0.791
Early stopping at epoch 198 for fold 2
Seed 42, Fold 2 Val Acc: 0.791
Early stopping at epoch 180 for fold 3
Seed 42, Fold 3 Val Acc: 0.779
Early stopping at epoch 183 for fold 4
Seed 42, Fold 4 Val Acc: 0.762
Early stopping at epoch 190 for fold 5
Seed 42, Fold 5 Val Acc: 0.761
Seed 42 Results: Mean Val Acc: 0.777, Time: 40.396 seconds, Memory: 3.783 MB, GPU Memory: 603.980 MB
Early stopping at epoch 180 for fold 1
Seed 123, Fold 1 Val Acc: 0.764
Early stopping at epoch 190 for fold 2
Seed 123, Fold 2 Val Acc: 0.779
Early stopping at epoch 179 for fold 3
Seed 123, Fold 3 Val Acc: 0.786
Seed 123, Fold 4 Val Acc: 0.779
Early stopping at epoch 187 for fold 5
Seed 123, Fold 5 Val Acc: 0.776
Seed 123 Results: Mean Val Acc: 0.777, Time: 37.863 seconds, Memory: 0.361 MB, GPU Memory: 603.980 MB
Early stopping at epoch 

### Pubmed

In [29]:
dataset_sparse = Planetoid(root="/data/Zeyu/Pooling", name='PubMed')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=3, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])
Seed 42, Fold 1 Val Acc: 0.882
Seed 42, Fold 2 Val Acc: 0.881
Seed 42, Fold 3 Val Acc: 0.866
Seed 42, Fold 4 Val Acc: 0.880
Seed 42, Fold 5 Val Acc: 0.879
Seed 42 Results: Mean Val Acc: 0.878, Time: 47.855 seconds, Memory: 13.105 MB, GPU Memory: 782.238 MB
Seed 123, Fold 1 Val Acc: 0.874
Seed 123, Fold 2 Val Acc: 0.870
Seed 123, Fold 3 Val Acc: 0.883
Seed 123, Fold 4 Val Acc: 0.880
Seed 123, Fold 5 Val Acc: 0.880
Seed 123 Results: Mean Val Acc: 0.877, Time: 50.107 seconds, Memory: 0.821 MB, GPU Memory: 782.238 MB
Seed 456, Fold 1 Val Acc: 0.886
Seed 456, Fold 2 Val Acc: 0.874
Seed 456, Fold 3 Val Acc: 0.880
Seed 456, Fold 4 Val Acc: 0.881
Seed 456, Fold 5 Val Acc: 0.870
Seed 456 Results: Mean Val Acc: 0.878, Time: 44.027 seconds, Memory: 0.820 MB, GPU Memory: 782.238 MB
Seed 789, Fold 1 Val Acc: 0.879
Seed 789, Fold 2 Val Acc: 0.878
Seed 789, Fold 3 Val Acc: 0.878
Seed 789, F

### Cornell

In [30]:
from torch_geometric.datasets import WebKB
dataset_sparse = WebKB(root="/data/Zeyu/Pooling", name='Cornell')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=1, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[183, 1703], edge_index=[2, 298], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])
Seed 42, Fold 1 Val Acc: 0.811
Seed 42, Fold 2 Val Acc: 0.757
Seed 42, Fold 3 Val Acc: 0.838
Seed 42, Fold 4 Val Acc: 0.833
Seed 42, Fold 5 Val Acc: 0.694
Seed 42 Results: Mean Val Acc: 0.787, Time: 32.437 seconds, Memory: 0.295 MB, GPU Memory: 782.238 MB
Seed 123, Fold 1 Val Acc: 0.838
Seed 123, Fold 2 Val Acc: 0.703
Seed 123, Fold 3 Val Acc: 0.703
Seed 123, Fold 4 Val Acc: 0.861
Seed 123, Fold 5 Val Acc: 0.861
Seed 123 Results: Mean Val Acc: 0.793, Time: 33.114 seconds, Memory: 0.279 MB, GPU Memory: 782.238 MB
Seed 456, Fold 1 Val Acc: 0.811
Seed 456, Fold 2 Val Acc: 0.757
Early stopping at epoch 188 for fold 3
Seed 456, Fold 3 Val Acc: 0.784
Seed 456, Fold 4 Val Acc: 0.833
Seed 456, Fold 5 Val Acc: 0.861
Seed 456 Results: Mean Val Acc: 0.809, Time: 32.043 seconds, Memory: 0.277 MB, GPU Memory: 782.238 MB
Seed 789, Fold 1 Val Acc: 0.784
Seed 789, Fold 2 Val Acc: 0.730
Seed

### Texas

In [31]:
from torch_geometric.datasets import WebKB
dataset_sparse = WebKB(root="/data/Zeyu/Pooling", name='texas')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=1, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[183, 1703], edge_index=[2, 325], y=[183], train_mask=[183, 10], val_mask=[183, 10], test_mask=[183, 10])
Seed 42, Fold 1 Val Acc: 0.811
Seed 42, Fold 2 Val Acc: 0.838
Seed 42, Fold 3 Val Acc: 0.811
Early stopping at epoch 180 for fold 4
Seed 42, Fold 4 Val Acc: 0.750
Early stopping at epoch 199 for fold 5
Seed 42, Fold 5 Val Acc: 0.750
Seed 42 Results: Mean Val Acc: 0.792, Time: 30.321 seconds, Memory: 0.297 MB, GPU Memory: 782.238 MB
Seed 123, Fold 1 Val Acc: 0.811
Seed 123, Fold 2 Val Acc: 0.865
Seed 123, Fold 3 Val Acc: 0.838
Early stopping at epoch 183 for fold 4
Seed 123, Fold 4 Val Acc: 0.833
Seed 123, Fold 5 Val Acc: 0.750
Seed 123 Results: Mean Val Acc: 0.819, Time: 34.745 seconds, Memory: 0.280 MB, GPU Memory: 782.238 MB
Seed 456, Fold 1 Val Acc: 0.838
Early stopping at epoch 196 for fold 2
Seed 456, Fold 2 Val Acc: 0.784
Seed 456, Fold 3 Val Acc: 0.838
Seed 456, Fold 4 Val Acc: 0.833
Seed 456, Fold 5 Val Acc: 0.778
Seed 456 Results: Mean Val Acc: 0.814, Time: 32.229 s

### Wisconsin

In [33]:
from torch_geometric.datasets import WebKB
dataset_sparse = WebKB(root="/data/Zeyu/Pooling", name='wisconsin')
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])

import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=0, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[251, 1703], edge_index=[2, 515], y=[251], train_mask=[251, 10], val_mask=[251, 10], test_mask=[251, 10])
Early stopping at epoch 188 for fold 1
Seed 42, Fold 1 Val Acc: 0.804
Early stopping at epoch 173 for fold 2
Seed 42, Fold 2 Val Acc: 0.840
Seed 42, Fold 3 Val Acc: 0.800
Seed 42, Fold 4 Val Acc: 0.820
Seed 42, Fold 5 Val Acc: 0.800
Seed 42 Results: Mean Val Acc: 0.813, Time: 23.927 seconds, Memory: 0.297 MB, GPU Memory: 713.032 MB
Early stopping at epoch 176 for fold 1
Seed 123, Fold 1 Val Acc: 0.745
Seed 123, Fold 2 Val Acc: 0.840
Seed 123, Fold 3 Val Acc: 0.940
Seed 123, Fold 4 Val Acc: 0.780
Seed 123, Fold 5 Val Acc: 0.860
Seed 123 Results: Mean Val Acc: 0.833, Time: 26.273 seconds, Memory: 0.283 MB, GPU Memory: 715.129 MB
Seed 456, Fold 1 Val Acc: 0.824
Early stopping at epoch 194 for fold 2
Seed 456, Fold 2 Val Acc: 0.840
Seed 456, Fold 3 Val Acc: 0.900
Seed 456, Fold 4 Val Acc: 0.820
Early stopping at epoch 178 for fold 5
Seed 456, Fold 5 Val Acc: 0.760
Seed 456 Resul

### Github

In [36]:
from torch_geometric.datasets import GitHub
dataset_sparse = GitHub(root="/data/Zeyu/Pooling/GitHub")
dataset_sparse = dataset_sparse.shuffle()
print(dataset_sparse[0])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import time
import tracemalloc
import torch
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
import torch.nn as nn
from sklearn.model_selection import KFold
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv, TopKPooling
from torch_geometric.data import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.transforms import ToUndirected
from torch.nn import Linear
import torch.optim as optim
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import to_dense_batch
from sklearn.model_selection import KFold
import numpy as np
import random
from typing import Callable, Optional, Union

dataset = dataset_sparse
graph = dataset[0]
num_classes = dataset.num_classes
in_channels = dataset.num_features
hidden_channels = 64
out_channels = num_classes
depth = 2
pool_ratios = [0.5, 0.5]

class HierarchicalGCN_KMIS(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, depth, pool_ratios, act=F.relu, sum_res=False):
        super(HierarchicalGCN_KMIS, self).__init__()
        assert depth >= 1
        self.in_channels = in_channels
        self.hidden_channels = hidden_channels
        self.out_channels = out_channels
        self.depth = depth
        self.pool_ratios = pool_ratios
        self.act = act
        self.sum_res = sum_res

        channels = self.hidden_channels

        self.down_convs = torch.nn.ModuleList()
        self.pools = torch.nn.ModuleList()
        self.down_convs.append(SAGEConv(self.in_channels, channels))
        for i in range(self.depth):
            self.pools.append(KMISPooling(64, k=6, aggr_x='sum'))
            self.down_convs.append(SAGEConv(channels, channels))

        in_channels = channels if sum_res else 2 * channels

        self.up_convs = torch.nn.ModuleList()
        for i in range(self.depth):
            self.up_convs.append(SAGEConv(in_channels, channels))
        self.up_convs.append(SAGEConv(channels, self.out_channels))

    def forward(self, x, edge_index, batch=None):
        x, edge_index = x.to(device), edge_index.to(device)
        if batch is None:
            batch = edge_index.new_zeros(x.size(0))
        if batch is not None:
            batch = batch.to(device)

        x = F.dropout(x, p=0.5, training=self.training)
        x = self.down_convs[0](x, edge_index)
        x = self.act(x)

        xs = [x]
        edge_indices = [edge_index]
        perms = []

        for i in range(1, self.depth + 1):
            x, edge_index, _, batch, _, cluster, perm = self.pools[i - 1](x, edge_index, batch=batch)
            x = self.down_convs[i](x, edge_index)
            x = self.act(x)

            if i < self.depth:
                xs.append(x)
                edge_indices.append(edge_index)
            perms.append(perm)

        for i in range(self.depth):
            j = self.depth - 1 - i

            res = xs[j]
            edge_index = edge_indices[j]
            perm = perms[j]

            up = torch.zeros_like(res)
            up[perm] = x
            x = res + up if self.sum_res else torch.cat((res, up), dim=-1)
            x = self.up_convs[i](x, edge_index)
            x = self.act(x)
        x = self.up_convs[-1](x, edge_index)

        return x

def train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask, n_epochs=200, patience=150, min_delta=0.0001):
    best_val_acc = 0
    patience_counter = 0
    model.to(device)
    graph = graph.to(device)  

    for epoch in range(1, n_epochs + 1):
        model.train()
        optimizer.zero_grad()
        out = model(graph.x, graph.edge_index)
        loss = criterion(out[train_mask], graph.y[train_mask])
        loss.backward()
        optimizer.step()

        val_acc = eval_node_classifier(model, graph, val_mask)

        # Early stopping logic
        if val_acc > best_val_acc + min_delta:
            best_val_acc = val_acc
            patience_counter = 0  # Reset patience counter if improvement
        else:
            patience_counter += 1  # Increment patience counter if no improvement

        if patience_counter >= patience:
            print(f'Early stopping at epoch {epoch} for fold {fold + 1}')
            break

    return model, best_val_acc

def eval_node_classifier(model, graph, mask):
    model.eval()
    pred = model(graph.x, graph.edge_index).argmax(dim=1)
    correct = (pred[mask] == graph.y[mask]).sum()
    acc = int(correct) / int(mask.sum())
    return acc

# Cross-validation
kf = KFold(n_splits=5, shuffle=True)


seeds = [42, 123, 456, 789, 101]
results = []
val_accuracies_list = []
times = []
memories = []
gpu_memories = []

for seed in seeds:
    graph = graph.to(device)
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

    val_accuracies = []
    start_time = time.time()
    tracemalloc.start()

    for fold, (train_index, test_index) in enumerate(kf.split(graph.x)):
        #print(f'Seed {seed}, Fold {fold + 1}')

        # Reset model
        model = HierarchicalGCN_KMIS(in_channels, hidden_channels, out_channels, depth, pool_ratios).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = nn.CrossEntropyLoss()

        # Create train and test masks
        train_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)
        test_mask = torch.zeros(graph.num_nodes, dtype=torch.bool)

        train_mask[train_index] = True
        test_mask[test_index] = True

        val_mask = test_mask  # In cross-validation, we use test_mask as val_mask

        model, best_val_acc = train_node_classifier(model, graph, optimizer, criterion, train_mask, val_mask)

        val_accuracies.append(best_val_acc)
        print(f'Seed {seed}, Fold {fold + 1} Val Acc: {best_val_acc:.3f}')

    mean_val_acc = np.mean(val_accuracies)
    end_time = time.time()
    current, peak = tracemalloc.get_traced_memory()
    tracemalloc.stop()
    memory_usage = peak / 10**6  # Convert to MB

    if torch.cuda.is_available():
        gpu_memory_usage = torch.cuda.memory_reserved(device) / 10**6  # Convert to MB
    else:
        gpu_memory_usage = 0

    elapsed_time = end_time - start_time
    results.append({
        'seed': seed,
        'mean_val_acc': mean_val_acc,
        'time': elapsed_time,
        'memory': memory_usage,
        'gpu_memory': gpu_memory_usage
    })
    print(f'Seed {seed} Results: Mean Val Acc: {mean_val_acc:.3f}, Time: {elapsed_time:.3f} seconds, Memory: {memory_usage:.3f} MB, GPU Memory: {gpu_memory_usage:.3f} MB')

for result in results:
    print(result)

mean_val_acc_values = [result['mean_val_acc'] for result in results]

# 计算总平均值和标准差
total_mean_val_acc = np.mean(mean_val_acc_values)
standard_deviation = np.std(mean_val_acc_values)

print(f"Total Mean Val Acc: {total_mean_val_acc:.2f}")
print(f"Standard Deviation: {standard_deviation:.2f}")

Data(x=[37700, 128], edge_index=[2, 578006], y=[37700])
Seed 42, Fold 1 Val Acc: 0.867
Seed 42, Fold 2 Val Acc: 0.880
Seed 42, Fold 3 Val Acc: 0.866
Seed 42, Fold 4 Val Acc: 0.871
Seed 42, Fold 5 Val Acc: 0.873
Seed 42 Results: Mean Val Acc: 0.871, Time: 70.116 seconds, Memory: 1.435 MB, GPU Memory: 719.323 MB
Seed 123, Fold 1 Val Acc: 0.871
Seed 123, Fold 2 Val Acc: 0.867
Seed 123, Fold 3 Val Acc: 0.869
Seed 123, Fold 4 Val Acc: 0.873
Seed 123, Fold 5 Val Acc: 0.874
Seed 123 Results: Mean Val Acc: 0.870, Time: 67.111 seconds, Memory: 1.411 MB, GPU Memory: 719.323 MB
Seed 456, Fold 1 Val Acc: 0.866
Seed 456, Fold 2 Val Acc: 0.878
Seed 456, Fold 3 Val Acc: 0.868
Seed 456, Fold 4 Val Acc: 0.872
Seed 456, Fold 5 Val Acc: 0.870
Seed 456 Results: Mean Val Acc: 0.871, Time: 67.856 seconds, Memory: 1.413 MB, GPU Memory: 719.323 MB
Seed 789, Fold 1 Val Acc: 0.869
Seed 789, Fold 2 Val Acc: 0.874
Seed 789, Fold 3 Val Acc: 0.870
Seed 789, Fold 4 Val Acc: 0.869
Seed 789, Fold 5 Val Acc: 0.869
Seed

In [9]:
import deepchem as dc
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from deepchem.feat import MolGraphConvFeaturizer
from deepchem.data import NumpyDataset

# 加载QM7数据集
qm7_dataset = "/data/Zeyu/Pooling/Graph_Pooling_Benchmark/Regression/datasets/qm7.csv"
data = pd.read_csv(qm7_dataset)

# 提取分子SMILES表示
smiles = data['smiles'].values

# 使用DeepChem的MolGraphConvFeaturizer将分子SMILES转换为图形特征
featurizer = MolGraphConvFeaturizer()
mols = featurizer.featurize(smiles)

Failed to featurize datapoint 0, C. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.


[22:06:02] WARNING: not removing hydrogen atom without neighbors
Failed to featurize datapoint 5505, [H]. Appending empty array
Exception message: More than one atom should be present in the molecule for this featurizer to work.
[22:06:02] WARNING: not removing hydrogen atom without neighbors
[22:06:02] WARNING: not removing hydrogen atom without neighbors
[22:06:03] WARNING: not removing hydrogen atom without neighbors
[22:06:03] WARNING: not removing hydrogen atom without neighbors
[22:06:03] WARNING: not removing hydrogen atom without neighbors
[22:06:03] WARNING: not removing hydrogen atom without neighbors
Exception message: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (6834,) + inhomogeneous part.


In [17]:
def graphdata_to_nx(graph_data):
    G = nx.Graph()
    num_nodes = graph_data.node_features.shape[0]
    G.add_nodes_from(range(num_nodes))
    for i in range(graph_data.edge_index.shape[1]):
        source = graph_data.edge_index[0, i]
        target = graph_data.edge_index[1, i]
        G.add_edge(source, target)
    return G

# 初始化累积变量
total_nodes = 0
total_edges = 0
total_clustering_coefficient = 0
total_diameter = 0
total_degree = 0
connected_graph_count = 0
total_graph_count = 0

# 计算并累积图形特性
for i, mol in enumerate(mols):
    if isinstance(mol, dc.feat.graph_data.GraphData):  # 确保 mol 是 GraphData 对象
        if mol.node_features.size == 0:  # 跳过空图
            continue
        G = graphdata_to_nx(mol)
        num_nodes = G.number_of_nodes()
        num_edges = G.number_of_edges()
        
        total_nodes += num_nodes
        total_edges += num_edges
        total_clustering_coefficient += nx.average_clustering(G)
        total_degree += sum(dict(G.degree()).values()) / num_nodes  # 累积平均度数
        
        if nx.is_connected(G):
            total_diameter += nx.diameter(G)
            connected_graph_count += 1  # 仅对连通图计算直径
        total_graph_count += 1

if total_graph_count > 0:
    avg_nodes = total_nodes / total_graph_count
    avg_edges = total_edges / total_graph_count
    avg_clustering_coefficient = total_clustering_coefficient / total_graph_count
    avg_diameter = total_diameter / connected_graph_count if connected_graph_count > 0 else float('nan')
    avg_degree = total_degree / total_graph_count

    # 输出保留两位小数
    print(f"Average number of nodes: {avg_nodes:.2f}")
    print(f"Average number of edges: {avg_edges:.2f}")
    print(f"Average clustering coefficient: {avg_clustering_coefficient:.2f}")
    print(f"Average diameter: {avg_diameter:.2f}")
    print(f"Average degree: {avg_degree:.2f}")
else:
    print("No valid graphs found in the dataset.")

Average number of nodes: 6.79
Average number of edges: 6.44
Average clustering coefficient: 0.06
Average diameter: 4.21
Average degree: 1.89
